## Interactive Dashboard with Ploty Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

In [71]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import requests

In [72]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
response = requests.get(url)

# Save the content to a file
with open('spacex_launch_dash.csv', 'wb') as file:
    file.write(response.content)

In [73]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
response = requests.get(url)

# Save the content to a file
with open('spacex_dash_app.py', 'wb') as file:
    file.write(response.content)
# filen er kopieret ind i følgende ramme

In [74]:
# Inlæs data til en pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


In [75]:
# dan liste med kolonnenavne til brug for valg af site
spacex_df.columns

Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')

In [76]:
# Dan en dash applikation
app = dash.Dash(__name__)

In [77]:
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Pie chart for all sites
        fig = px.pie(
            spacex_df,
            values='class',
            names='Launch Site',
            title='Total Success Launches for All Sites'
        )
    else:
        # Filtered pie chart for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(
            filtered_df,
            names='class',
            title=f'Total Success Launches for Site {entered_site}'
        )
    return fig

In [78]:
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")]
)
def get_scatter(entered_site,chosen_payload):
    if entered_site == 'ALL':
        # Pie chart for all sites
        fig = px.scatter(
            spacex_df,
            x='Payload Mass (kg)',
            y='class',
            color="Booster Version Category",
            title='Correlation between payload and success for all launch sites'
        )
    else:
        # Filtered pie chart for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color="Booster Version Category",
            title=f'Correlation between payload, booster version and success for launch site {entered_site}'
        )
    return fig

In [79]:
# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # TASK 1: Add a dropdown list to enable Launch Site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
        ],
        value='ALL',
        placeholder="Select a Launch Site",
        searchable=True
    ),
    html.Br(),
    
    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    
    html.P("Payload range (Kg):", style={"color": "white"}),

    # Payload slider
    dcc.RangeSlider(id='payload-slider',
    min=0, max=10000, step=1000,
    marks={i: {'label': str(i), 'style': {'color': 'white'}} for i in range(0, 10000, 1000)},
    value=[min_payload, max_payload],
    tooltip={"always_visible": True, "placement": "bottom"},
    updatemode='drag',
    className='slider-style'),
    
    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
# (To be implemented)

# Run the app
if __name__ == '__main__':
    app.run_server()
